In [1]:
## conda environment Weather_Prediction

import os
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
pd.set_option('display.max_columns', None)

In [2]:
## get the variables

PATH_1=os.path.join('..','..','..','Weather_Data','ERA5','2009','era_2009.nc')
humidity_data = xr.open_dataset(PATH_1) ## variables
humidity_data

<xarray.Dataset> Size: 542MB
Dimensions:         (valid_time: 2920, pressure_level: 1, latitude: 100,
                     longitude: 232)
Coordinates:
    number          int64 8B ...
  * valid_time      (valid_time) datetime64[ns] 23kB 2009-01-01 ... 2009-12-3...
  * pressure_level  (pressure_level) float64 8B 1e+03
  * latitude        (latitude) float64 800B 49.25 49.0 48.75 ... 25.0 24.75 24.5
  * longitude       (longitude) float64 2kB -124.8 -124.6 ... -67.35 -67.1
    expver          (valid_time) <U4 47kB ...
Data variables:
    r               (valid_time, pressure_level, latitude, longitude) float32 271MB ...
    q               (valid_time, pressure_level, latitude, longitude) float32 271MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T21:13 GRIB to CDM+CF via cfgrib-0.9.1...

In [3]:
humidity_data= humidity_data.drop_vars('number') ## not sure what these two are
humidity_data= humidity_data.drop_vars('expver')
humidity_data

<xarray.Dataset> Size: 542MB
Dimensions:         (valid_time: 2920, pressure_level: 1, latitude: 100,
                     longitude: 232)
Coordinates:
  * valid_time      (valid_time) datetime64[ns] 23kB 2009-01-01 ... 2009-12-3...
  * pressure_level  (pressure_level) float64 8B 1e+03
  * latitude        (latitude) float64 800B 49.25 49.0 48.75 ... 25.0 24.75 24.5
  * longitude       (longitude) float64 2kB -124.8 -124.6 ... -67.35 -67.1
Data variables:
    r               (valid_time, pressure_level, latitude, longitude) float32 271MB ...
    q               (valid_time, pressure_level, latitude, longitude) float32 271MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T21:13 GRIB to CDM+CF via cfgrib-0.9.1...

In [4]:
humidity_data.to_dataframe()

r         q
valid_time          pressure_level latitude longitude                     
2009-01-01 00:00:00 1000.0         49.25    -124.85    89.820335  0.003500
                                            -124.60    83.730492  0.003426
                                            -124.35    74.187523  0.003272
                                            -124.10    65.841820  0.003065
                                            -123.85    59.757835  0.002891
...                                                          ...       ...
2009-12-31 21:00:00 1000.0         24.50    -68.10     57.739704  0.008764
                                            -67.85     55.542439  0.008561
                                            -67.60     54.036579  0.008393
                                            -67.35     53.311970  0.008347
                                            -67.10     54.770954  0.008567

[67744000 rows x 2 columns]

In [5]:
## resample the data to 1 year average

year_avg = humidity_data.resample(valid_time='1YE').mean(dim='valid_time')
year_avg

<xarray.Dataset> Size: 188kB
Dimensions:         (valid_time: 1, pressure_level: 1, latitude: 100,
                     longitude: 232)
Coordinates:
  * pressure_level  (pressure_level) float64 8B 1e+03
  * latitude        (latitude) float64 800B 49.25 49.0 48.75 ... 25.0 24.75 24.5
  * longitude       (longitude) float64 2kB -124.8 -124.6 ... -67.35 -67.1
  * valid_time      (valid_time) datetime64[ns] 8B 2009-12-31
Data variables:
    r               (valid_time, pressure_level, latitude, longitude) float32 93kB ...
    q               (valid_time, pressure_level, latitude, longitude) float32 93kB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T21:13 GRIB to CDM+CF via cfgrib-0.9.1...

In [6]:
year_avg.to_dataframe()

r         q
valid_time pressure_level latitude longitude                     
2009-12-31 1000.0         49.25    -124.85    79.379288  0.005912
                                   -124.60    76.653915  0.005932
                                   -124.35    74.105186  0.005803
                                   -124.10    71.631821  0.005685
                                   -123.85    68.861572  0.005678
...                                                 ...       ...
                          24.50    -68.10     76.788277  0.014095
                                   -67.85     76.730469  0.014075
                                   -67.60     76.678391  0.014055
                                   -67.35     76.605873  0.014034
                                   -67.10     76.575668  0.014015

[23200 rows x 2 columns]

In [7]:
year_avg = year_avg.squeeze(dim="pressure_level") ## drop the pressure level dimension
year_avg

<xarray.Dataset> Size: 188kB
Dimensions:         (valid_time: 1, latitude: 100, longitude: 232)
Coordinates:
    pressure_level  float64 8B 1e+03
  * latitude        (latitude) float64 800B 49.25 49.0 48.75 ... 25.0 24.75 24.5
  * longitude       (longitude) float64 2kB -124.8 -124.6 ... -67.35 -67.1
  * valid_time      (valid_time) datetime64[ns] 8B 2009-12-31
Data variables:
    r               (valid_time, latitude, longitude) float32 93kB 79.38 ... ...
    q               (valid_time, latitude, longitude) float32 93kB 0.005912 ....
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T21:13 GRIB to CDM+CF via cfgrib-0.9.1...

In [8]:
SHAPE_PATH=os.path.join('..','..','..','Shapefiles','county_shapefiles','2009_county_shapefile','tl_2009_us_county.shp')
county_gdf = gpd.read_file(SHAPE_PATH)
county_gdf

,STATEFP,COUNTYFP,COUNTYNS,CNTYIDFP,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,16,035,00395697,16035,Clearwater,Clearwater County,06,H1,G4020,None,None,None,A,6364465317,79541433,+46.6725699,-115.6535002,"POLYGON ((-115.97504 46.29399, -115.97542 46.2..."
1,16,033,00399755,16033,Clark,Clark County,06,H1,G4020,None,None,None,A,4569211245,2476849,+44.2862511,-112.3611201,"POLYGON ((-112.73322 44.49991, -112.73152 44.5..."
2,16,025,00399408,16025,Camas,Camas County,06,H1,G4020,None,None,None,A,2782930314,11535337,+43.4633957,-114.8044265,"POLYGON ((-114.46408 43.32532, -114.46307 43.3..."
3,16,049,00395699,16049,Idaho,Idaho County,06,H1,G4020,None,None,None,A,21956209975,67333453,+45.8492367,-115.4673755,"POLYGON ((-116.52984 45.53828, -116.52981 45.5..."
4,16,019,00395407,16019,Bonneville,Bonneville County,06,H1,G4020,292,26820,None,A,4832931620,89483679,+43.3951707,-111.6218783,"POLYGON ((-112.52009 43.54467, -112.52010 43.5..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3229,48,069,01383820,48069,Castro,Castro County,06,H1,G4020,None,None,None,A,2316590566,12585110,+34.5336206,-102.2587856,"POLYGON ((-102.12045 34.74766, -102.12042 34.7..."
3230,48,127,01383849,48127,Dimmit,Dimmit County,06,H1,G4020,None,None,None,A,3441683133,14576058,+28.4235871,-099.7658713,"POLYGON ((-100.00815 28.19871, -100.01229 28.1..."
3231,48,271,01383921,48271,Kinney,Kinney County,06,H1,G4020,None,None,None,A,3522514289,13125084,+29.3470864,-100.4177003,"POLYGON ((-100.69998 29.49896, -100.69999 29.5..."
3232,48,177,01383874,48177,Gonzales,Gonzales County,06,H1,G4020,None,None,None,A,2762705077,8200950,+29.4687114,-097.4777298,"POLYGON ((-97.77281 29.26596, -97.77331 29.266..."


In [9]:
geometry_column=county_gdf['geometry']
geometry_column

0       POLYGON ((-115.97504 46.29399, -115.97542 46.2...
1       POLYGON ((-112.73322 44.49991, -112.73152 44.5...
2       POLYGON ((-114.46408 43.32532, -114.46307 43.3...
3       POLYGON ((-116.52984 45.53828, -116.52981 45.5...
4       POLYGON ((-112.52009 43.54467, -112.52010 43.5...
                              ...                        
3229    POLYGON ((-102.12045 34.74766, -102.12042 34.7...
3230    POLYGON ((-100.00815 28.19871, -100.01229 28.1...
3231    POLYGON ((-100.69998 29.49896, -100.69999 29.5...
3232    POLYGON ((-97.77281 29.26596, -97.77331 29.266...
3233    POLYGON ((-96.66181 29.85619, -96.65864 29.859...
Name: geometry, Length: 3234, dtype: geometry

In [10]:
lat_lon=geometry_column.get_coordinates(index_parts=True)
lat_lon

x          y
0    0    -115.975042  46.293987
     1    -115.975423  46.294056
     2    -115.975768  46.294209
     3    -115.975971  46.294355
     4    -115.976267  46.294495
...               ...        ...
3233 3483  -96.665372  29.852009
     3484  -96.665151  29.852267
     3485  -96.662979  29.854690
     3486  -96.661906  29.856084
     3487  -96.661814  29.856193

[7629005 rows x 2 columns]

In [11]:
def single_county(var):
    
    ''' Function to interpolate the values in a single county by taking a 100 or less latitude and longitude pair.  
        Args:
        --------
             var (int): The index of the county in the shapefile.

        Returns:
        --------
            county_df: Dataframe consisting the values of the variables interpolated in the county.
    ''' 
    longitude= lat_lon.loc[(var), 'x']  # get the longitude 
    extract_val=len(longitude)        ## find number of longitude that a shapfile has

    ## if number of longitude or latitude is less than 100, that many lat-lon pairs will be extracted, if not, a 100 values
    if extract_val < 100:
        extract_val=extract_val
    else:
        extract_val=100

    longitude=longitude[:extract_val]  ## extract first 100 values or values less than 100
    
    latitude= lat_lon.loc[(var), 'y']   # get the latitude values
    latitude=latitude[:extract_val]    ## extract first 100 values or values less than 100

    lat_list=latitude.tolist()
    lon_list=longitude.tolist()

    ## find the corresponding values of the variables in the finer grid

    year_avg_finer= year_avg.interp(longitude=lon_list, latitude=lat_list)
    year_avg_finer
    
    df = year_avg_finer.squeeze(dim="valid_time") ## remove the valid_time dimension, note that this will appear as a column
                                                  ## after removed

    ## get only the 100 (or less) pair of latitude and longitude from the lat and lon list from the 100,000 (or less) rows
    get_vals=[]

    for i in range(0, extract_val):
        row=df.isel(latitude=[i], longitude=[i])
        row_df=row.to_dataframe()
        get_vals.append(row_df)

    initial_df=pd.concat(get_vals)
    initial_df=initial_df.drop('valid_time', axis=1) ## remove the valid_time column 
    
    mean=initial_df.mean() ## find the average of the 100 (or less) values
    average_df = pd.DataFrame(mean).T 

    county_df= average_df.convert_dtypes() ## the dtype of the mean values will be object, so convert that to float
    county_df
    
    return county_df

In [12]:
%%time

## include all counties

df_list=[]

for i in range(0,3234): ## A total of 3234 counties. index in shape file starts from 0 and ends in 3234
    try:
        df_list.append(single_county(i))
    except:
        pass

CPU times: total: 6min 25s
Wall time: 6min 32s


In [13]:
concatenate_df=pd.concat(df_list)
final_df=concatenate_df.reset_index(drop=True)
final_df = final_df.drop('pressure_level', axis=1)
final_df

,r,q
0,61.984339,0.004735
1,66.100985,0.003631
2,57.299841,0.003881
3,58.28394,0.004272
4,53.964966,0.003978
...,...,...
3229,48.476964,0.006301
3230,54.379006,0.010021
3231,54.384591,0.009164
3232,62.924777,0.010633


In [14]:
## lets first create a fips column in the original shape file

## lets create the fips column in the dataframe which is the sum of state and county both as dtype string.
## Note: not a numerical sum
## This step is necessary because there can be several counties with idential names. 

county_gdf['fips']=county_gdf['STATEFP'] + county_gdf['COUNTYFP']
county_gdf.dtypes
## convert the data type
county_gdf['fips']=county_gdf['fips'].astype(str).astype(int)
county_gdf.dtypes

STATEFP       object
COUNTYFP      object
COUNTYNS      object
CNTYIDFP      object
NAME          object
NAMELSAD      object
LSAD          object
CLASSFP       object
MTFCC         object
CSAFP         object
CBSAFP        object
METDIVFP      object
FUNCSTAT      object
ALAND          int64
AWATER         int64
INTPTLAT      object
INTPTLON      object
geometry    geometry
fips           int32
dtype: object

In [16]:
county_gdf = county_gdf.drop(['STATEFP','COUNTYFP','COUNTYNS','CNTYIDFP','NAMELSAD',
                              'LSAD','CLASSFP','MTFCC','CSAFP','CBSAFP','METDIVFP',
                             'FUNCSTAT','ALAND','AWATER','INTPTLAT','INTPTLON'], axis=1)
county_gdf

,NAME,geometry,fips
0,Clearwater,"POLYGON ((-115.97504 46.29399, -115.97542 46.2...",16035
1,Clark,"POLYGON ((-112.73322 44.49991, -112.73152 44.5...",16033
2,Camas,"POLYGON ((-114.46408 43.32532, -114.46307 43.3...",16025
3,Idaho,"POLYGON ((-116.52984 45.53828, -116.52981 45.5...",16049
4,Bonneville,"POLYGON ((-112.52009 43.54467, -112.52010 43.5...",16019
...,...,...,...
3229,Castro,"POLYGON ((-102.12045 34.74766, -102.12042 34.7...",48069
3230,Dimmit,"POLYGON ((-100.00815 28.19871, -100.01229 28.1...",48127
3231,Kinney,"POLYGON ((-100.69998 29.49896, -100.69999 29.5...",48271
3232,Gonzales,"POLYGON ((-97.77281 29.26596, -97.77331 29.266...",48177


In [17]:
merged_df=pd.merge(county_gdf, final_df, left_index=True, right_index=True)
merged_df

,NAME,geometry,fips,r,q
0,Clearwater,"POLYGON ((-115.97504 46.29399, -115.97542 46.2...",16035,61.984339,0.004735
1,Clark,"POLYGON ((-112.73322 44.49991, -112.73152 44.5...",16033,66.100985,0.003631
2,Camas,"POLYGON ((-114.46408 43.32532, -114.46307 43.3...",16025,57.299841,0.003881
3,Idaho,"POLYGON ((-116.52984 45.53828, -116.52981 45.5...",16049,58.28394,0.004272
4,Bonneville,"POLYGON ((-112.52009 43.54467, -112.52010 43.5...",16019,53.964966,0.003978
...,...,...,...,...,...
3229,Castro,"POLYGON ((-102.12045 34.74766, -102.12042 34.7...",48069,48.476964,0.006301
3230,Dimmit,"POLYGON ((-100.00815 28.19871, -100.01229 28.1...",48127,54.379006,0.010021
3231,Kinney,"POLYGON ((-100.69998 29.49896, -100.69999 29.5...",48271,54.384591,0.009164
3232,Gonzales,"POLYGON ((-97.77281 29.26596, -97.77331 29.266...",48177,62.924777,0.010633


In [18]:
county_df=merged_df.dropna()
county_df

,NAME,geometry,fips,r,q
0,Clearwater,"POLYGON ((-115.97504 46.29399, -115.97542 46.2...",16035,61.984339,0.004735
1,Clark,"POLYGON ((-112.73322 44.49991, -112.73152 44.5...",16033,66.100985,0.003631
2,Camas,"POLYGON ((-114.46408 43.32532, -114.46307 43.3...",16025,57.299841,0.003881
3,Idaho,"POLYGON ((-116.52984 45.53828, -116.52981 45.5...",16049,58.28394,0.004272
4,Bonneville,"POLYGON ((-112.52009 43.54467, -112.52010 43.5...",16019,53.964966,0.003978
...,...,...,...,...,...
3229,Castro,"POLYGON ((-102.12045 34.74766, -102.12042 34.7...",48069,48.476964,0.006301
3230,Dimmit,"POLYGON ((-100.00815 28.19871, -100.01229 28.1...",48127,54.379006,0.010021
3231,Kinney,"POLYGON ((-100.69998 29.49896, -100.69999 29.5...",48271,54.384591,0.009164
3232,Gonzales,"POLYGON ((-97.77281 29.26596, -97.77331 29.266...",48177,62.924777,0.010633


In [19]:
type(county_df)

geopandas.geodataframe.GeoDataFrame

In [20]:
## convert into pandas dataframe without the geometry column
pd.options.display.float_format = '{:.15f}'.format ## see 15 decimal places of the numbers
county_var=pd.DataFrame(county_df.drop(columns='geometry')) 
county_var

,NAME,fips,r,q
0,Clearwater,16035,61.984339396273882,0.004735322729210
1,Clark,16033,66.100985145914976,0.003630530572376
2,Camas,16025,57.299841195804468,0.003880671214170
3,Idaho,16049,58.283940184395263,0.004271987964606
4,Bonneville,16019,53.964965618329231,0.003978249607555
...,...,...,...,...
3229,Castro,48069,48.476963611459063,0.006300885882014
3230,Dimmit,48127,54.379006087601944,0.010020948072791
3231,Kinney,48271,54.384591478100837,0.009164113486763
3232,Gonzales,48177,62.924776898617104,0.010632521816516


In [21]:
county_var = county_var.sort_values(by='fips')
county_var

,NAME,fips,r,q
520,Autauga,1001,69.254996976143232,0.009958306855491
516,Baldwin,1003,74.628750963049129,0.011885168648705
522,Barbour,1005,69.986613194566218,0.010243629102019
501,Bibb,1007,71.056572625519820,0.009781784378499
535,Blount,1009,72.410443813328826,0.009430635424257
...,...,...,...,...
2392,Sweetwater,56037,56.543425908676141,0.003769271303067
2848,Teton,56039,66.831657246038731,0.003783448517526
1717,Uinta,56041,56.310133191534312,0.003757365436584
2863,Washakie,56043,55.740740369266888,0.003842903983282


In [22]:
county_var.to_pickle('era_2009.pkl')